In [ ]:
import time
import numpy as np
from memory_profiler import memory_usage

np.random.seed(42)


def get_minor(matrix, i, j):
    return [row[:j] + row[j+1:] for k, row in enumerate(matrix) if k != i]

def determinant(matrix):
    n = len(matrix)
    if n == 1:
        return matrix[0][0]
    if n == 2:
        return matrix[0][0]*matrix[1][1] - matrix[0][1]*matrix[1][0]
    return sum(
        (-1)**j * matrix[0][j] * determinant(get_minor(matrix, 0, j))
        for j in range(n)
    )

def cofactor_matrix(matrix):
    n = len(matrix)
    return [
        [(-1)**(i + j) * determinant(get_minor(matrix, i, j)) for j in range(n)]
        for i in range(n)
    ]

def transpose(matrix):
    return [list(row) for row in zip(*matrix)]

def adjugate_matrix(matrix):
    return transpose(cofactor_matrix(matrix))

def invert_matrix(matrix):
    det = determinant(matrix)
    if det == 0:
        raise ValueError("Zero determinant")
    adj = adjugate_matrix(matrix)
    return [[elem / det for elem in row] for row in adj]

# Генерація випадкових матриць
def generate_random_matrix(n):
    return np.random.uniform(1, 10, (n, n)).tolist()

# Обчислення похибки
def compute_error(A, A_inv):
    A_np = np.array(A)
    A_inv_np = np.array(A_inv)
    identity = np.eye(len(A_np))
    approx_I = A_np @ A_inv_np
    return np.linalg.norm(approx_I - identity)

def wrapper(func, *args):
    return func(*args)

if __name__ == '__main__':
    matrix_sizes = [3, 5, 10, 100, 1000]

    for size in matrix_sizes:
        A = generate_random_matrix(size)
        try:
            cond_number = np.linalg.cond(A)

            start_time = time.time()
            mem_usage, A_inv = memory_usage(
                (wrapper, (invert_matrix, A)),
                retval=True, max_iterations=1
            )
            end_time = time.time()

            execution_time = end_time - start_time
            peak_memory = max(mem_usage) - min(mem_usage)
            error = compute_error(A, A_inv)

            print(f"\nSize {size}x{size}")
            print(f"Condition number: {cond_number:.2e}")
            print(f"Execution time: {execution_time:.6f} seconds")
            print(f"Peak memory usage: {peak_memory:.6f} MiB")
            print(f"Error ||AA⁻¹ - I||: {error:.2e}")
            print(f"Inverse matrix {size}x{size}:")
            for row in A_inv:
                print(row)
        except Exception as e:
            print(f"\nSize {size}x{size}: FAILED — {e}")
